# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
#Importando bibliotecas
import pandas as pd
import numpy as np
import patsy
from patsy import dmatrix
import statsmodels.api as sm

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

## 1- Ajustando um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

##### Verificando as variáveis qualitativas

In [4]:
df.select_dtypes(include=['object', 'bool'])

,data_ref,sexo,posse_de_veiculo,posse_de_imovel,tipo_renda,educacao,estado_civil,tipo_residencia
0,2015-01-01,F,False,True,Empresário,Secundário,Solteiro,Casa
1,2015-01-01,M,True,True,Assalariado,Superior completo,Casado,Casa
2,2015-01-01,F,True,True,Empresário,Superior completo,Casado,Casa
3,2015-01-01,F,False,True,Servidor público,Superior completo,Casado,Casa
4,2015-01-01,M,True,False,Assalariado,Secundário,Solteiro,Governamental
...,...,...,...,...,...,...,...,...
14995,2016-03-01,F,False,True,Empresário,Secundário,Solteiro,Casa
14996,2016-03-01,F,False,True,Pensionista,Superior completo,Solteiro,Casa
14997,2016-03-01,F,True,True,Assalariado,Superior completo,Casado,Casa
14998,2016-03-01,M,True,False,Empresário,Superior completo,Casado,Casa


#### Verificando a moda das variáveis qualitativas

In [5]:
df_moda = {'Variáveis': ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao','estado_civil', 'tipo_residencia'],
           'Frequencia': [df.sexo.mode()[0], df.posse_de_veiculo.mode()[0], df.posse_de_imovel.mode()[0], df.tipo_renda.mode()[0], df.educacao.mode()[0],df. estado_civil.mode()[0], df.tipo_residencia.mode()[0]]
          }
df_moda= pd.DataFrame(df_moda)
df_moda

,Variáveis,Frequencia
0,sexo,F
1,posse_de_veiculo,False
2,posse_de_imovel,True
3,tipo_renda,Assalariado
4,educacao,Secundário
5,estado_civil,Casado
6,tipo_residencia,Casa


#### Criando a Design Matrix com o Patsy, mantendo a categoria mais frequente das variáveis qualitativas como casela de referência

In [6]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(0)) + C(posse_de_veiculo, Treatment(0)) +  C(posse_de_imovel, Treatment(1)) + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2)) + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1))', data = df)
x

DesignMatrix with shape (15000, 21)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(0))[T.M]',
     'C(posse_de_veiculo, Treatment(0))[T.True]',
     'C(posse_de_imovel, Treatment(1))[T.False]',
     'C(tipo_renda, Treatment(0))[T.Bolsista]',
     'C(tipo_renda, Treatment(0))[T.Empresário]',
     'C(tipo_renda, Treatment(0))[T.Pensionista]',
     'C(tipo_renda, Treatment(0))[T.Servidor público]',
     'C(educacao, Treatment(2))[T.Primário]',
     'C(educacao, Treatment(2))[T.Pós graduação]',
     'C(educacao, Treatment(2))[T.Superior completo]',
     'C(educacao, Treatment(2))[T.Superior incompleto]',
     'C(estado_civil, Treatment(0))[T.Separado]',
     'C(estado_civil, Treatment(0))[T.Solteiro]',
     'C(estado_civil, Treatment(0))[T.União]',
     'C(estado_civil, Treatment(0))[T.Viúvo]',
     'C(tipo_residencia, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia, Treatment(1))[T.Comunitário]',
     'C(tipo_residencia, Tre

#### Rodando o modelo e avaliando os parâmetros

In [7]:
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     144.7
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        11:46:16   Log-Likelihood:                -18100.
No. Observations:               15000   AIC:                         3.624e+04
Df Residuals:                   14979   BIC:                         3.640e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             7.9814      0.015    549.761      0.000       7.953       8.010
C(sexo, Treatment(0))[T.M]                            0.6971      0.015     45.019      0.000       0.667       0.727
C(posse_de_veiculo, Treatment(0))[T.True]             0.0103      0.015      0.696      0.486      -0.019       0.039
C(posse_de_imovel, Treatment(1))[T.False]            -0.0941      0.015     -6.477      0.000      -0.123      -0.066
C(tipo_renda, Treatment(0))[T.Bolsista]               0.3770      0.270      1.395      0.163      -0.153       0.907
C(tipo_renda, Treatment(0))[T.Empresário]             0.0680      0.017      4.088      0.000       0.035       0.101
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.1604      0.019     -8.278      0.000      -0.198      -0.122
C(tipo_renda, Treatment(0))[T.Servidor público]       0.2189      0.025      8.853      0.000       0.170       0.267
C(educacao, Treatment(2))[T.Primário]                 0.0178      0.064      0.278      0.781      -0.107       0.143
C(educacao, Treatment(2))[T.Pós graduação]           -0.1013      0.159     -0.636      0.525      -0.413       0.211
C(educacao, Treatment(2))[T.Superior completo]        0.0954      0.014      6.670      0.000       0.067       0.123
C(educacao, Treatment(2))[T.Superior incompleto]     -0.1178      0.035     -3.364      0.001      -0.186      -0.049
C(estado_civil, Treatment(0))[T.Separado]             0.0513      0.029      1.788      0.074      -0.005       0.107
C(estado_civil, Treatment(0))[T.Solteiro]            -0.0378      0.021     -1.812      0.070      -0.079       0.003
C(estado_civil, Treatment(0))[T.União]               -0.0641      0.026     -2.465      0.014      -0.115      -0.013
C(estado_civil, Treatment(0))[T.Viúvo]                0.0438      0.032      1.348      0.178      -0.020       0.107
C(tipo_residencia, Treatment(1))[T.Aluguel]          -0.1129      0.059     -1.919      0.055      -0.228       0.002
C(tipo_residencia, Treatment(1))[T.Com os pais]      -0.1306      0.033     -4.009      0.000      -0.194      -0.067
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.2398      0.102     -2.355      0.019      -0.439      -0.040
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1924      0.089      2.153      0.031       0.017       0.368
C(tipo_residencia, Treatment(1))[T.Governamental]     0.0460      0.039      1.177      0.239      -0.031       0.123
==============================================================================
Omnibus:                      151.788   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              173.551
Skew:         

In [8]:
R2_inicial = reg.rsquared
R2_inicial

0.16193446943715917

In [9]:
R2_ajust_inicial = reg.rsquared_adj
R2_ajust_inicial 

0.1608154821475366

In [10]:
AIC_inicial =reg.aic
AIC_inicial

36241.337462396375

## Removendo a variável menos significante

#### Removendo a variável menos significante - C(educacao, Treatment(2))[T.Primário]

Vamos criar uma nova variável 'educacao2' e unificar as classificações Primário e Secundário e aproveitar também vamos unificar Superior Completo e Pós graduação


In [11]:
df['educacao'].unique()

array(['Secundário', 'Superior completo', 'Superior incompleto',
       'Primário', 'Pós graduação'], dtype=object)

In [12]:
df.loc[ df['educacao'] == 'Primário', 'educacao2'] ='Primário/Secundário'
df.loc[ df['educacao'] == 'Secundário', 'educacao2'] = 'Primário/Secundário'
df.loc[ df['educacao'] == 'Superior completo', 'educacao2'] = 'Superior completo/Pós graduação'
df.loc[ df['educacao'] == 'Superior incompleto', 'educacao2'] = 'Superior incompleto'
df.loc[ df['educacao'] == 'Pós graduação', 'educacao2'] = 'Superior completo/Pós graduação'

df[['educacao','educacao2']].head()

,educacao,educacao2
0,Secundário,Primário/Secundário
1,Superior completo,Superior completo/Pós graduação
2,Superior completo,Superior completo/Pós graduação
3,Superior completo,Superior completo/Pós graduação
4,Secundário,Primário/Secundário


In [13]:
df['educacao2'].unique()

array(['Primário/Secundário', 'Superior completo/Pós graduação',
       'Superior incompleto'], dtype=object)

#### Criando a Design Matrix com o Patsy,usando 'educacao2' no lugar de 'educacao1'

In [14]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(0)) + C(posse_de_veiculo, Treatment(0)) +  C(posse_de_imovel, Treatment(1)) + C(tipo_renda, Treatment(0)) + C(educacao2, Treatment(2)) + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1))', data = df)
x

DesignMatrix with shape (15000, 19)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(0))[T.M]',
     'C(posse_de_veiculo, Treatment(0))[T.True]',
     'C(posse_de_imovel, Treatment(1))[T.False]',
     'C(tipo_renda, Treatment(0))[T.Bolsista]',
     'C(tipo_renda, Treatment(0))[T.Empresário]',
     'C(tipo_renda, Treatment(0))[T.Pensionista]',
     'C(tipo_renda, Treatment(0))[T.Servidor público]',
     'C(educacao2, Treatment(2))[T.Primário/Secundário]',
     'C(educacao2, Treatment(2))[T.Superior completo/Pós graduação]',
     'C(estado_civil, Treatment(0))[T.Separado]',
     'C(estado_civil, Treatment(0))[T.Solteiro]',
     'C(estado_civil, Treatment(0))[T.União]',
     'C(estado_civil, Treatment(0))[T.Viúvo]',
     'C(tipo_residencia, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia, Treatment(1))[T.Comunitário]',
     'C(tipo_residencia, Treatment(1))[T.Estúdio]',
     'C(tipo_residencia, Treatment(1))[T.Governamental]']

#### Rodando o modelo e avaliando os parâmetros

In [15]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     160.7
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        11:55:16   Log-Likelihood:                -18100.
No. Observations:               15000   AIC:                         3.624e+04
Df Residuals:                   14981   BIC:                         3.638e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         7.8636      0.036    217.906      0.000       7.793       7.934
C(sexo, Treatment(0))[T.M]                                        0.6974      0.015     45.045      0.000       0.667       0.728
C(posse_de_veiculo, Treatment(0))[T.True]                         0.0099      0.015      0.671      0.503      -0.019       0.039
C(posse_de_imovel, Treatment(1))[T.False]                        -0.0937      0.015     -6.453      0.000      -0.122      -0.065
C(tipo_renda, Treatment(0))[T.Bolsista]                           0.3780      0.270      1.399      0.162      -0.152       0.908
C(tipo_renda, Treatment(0))[T.Empresário]                         0.0676      0.017      4.064      0.000       0.035       0.100
C(tipo_renda, Treatment(0))[T.Pensionista]                       -0.1599      0.019     -8.259      0.000      -0.198      -0.122
C(tipo_renda, Treatment(0))[T.Servidor público]                   0.2193      0.025      8.871      0.000       0.171       0.268
C(educacao2, Treatment(2))[T.Primário/Secundário]                 0.1180      0.035      3.373      0.001       0.049       0.187
C(educacao2, Treatment(2))[T.Superior completo/Pós graduação]     0.2122      0.036      5.961      0.000       0.142       0.282
C(estado_civil, Treatment(0))[T.Separado]                         0.0516      0.029      1.801      0.072      -0.005       0.108
C(estado_civil, Treatment(0))[T.Solteiro]                        -0.0383      0.021     -1.833      0.067      -0.079       0.003
C(estado_civil, Treatment(0))[T.União]                           -0.0637      0.026     -2.450      0.014      -0.115      -0.013
C(estado_civil, Treatment(0))[T.Viúvo]                            0.0438      0.032      1.349      0.177      -0.020       0.107
C(tipo_residencia, Treatment(1))[T.Aluguel]                      -0.1125      0.059     -1.912      0.056      -0.228       0.003
C(tipo_residencia, Treatment(1))[T.Com os pais]                  -0.1301      0.033     -3.995      0.000      -0.194      -0.066
C(tipo_residencia, Treatment(1))[T.Comunitário]                  -0.2374      0.102     -2.335      0.020      -0.437      -0.038
C(tipo_residencia, Treatment(1))[T.Estúdio]                       0.1927      0.089      2.157      0.031       0.018       0.368
C(tipo_residencia, Treatment(1))[T.Governamental]                 0.0463      0.039      1.184      0.236      -0.030       0.123
==============================================================================
Omnibus:                      152.027   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        

Não houve alteração no R² contudo os valores de p_value para as variáveis referentes a educação ficaram melhores.

## Removendo as demais variáveis

Retiramos inicialmente a variável posse de veículo do modelo, estado civil

#### Vamos criar uma nova variável 'tipo_residencia2' e unificar as classificações Casa/Governamental

In [16]:
df['tipo_residencia'].unique()

array(['Casa', 'Governamental', 'Com os pais', 'Aluguel', 'Estúdio',
       'Comunitário'], dtype=object)

In [17]:
df.loc[ df['tipo_residencia'] == 'Casa', 'tipo_residencia2'] ='Casa/Governamental'
df.loc[ df['tipo_residencia'] == 'Governamental', 'tipo_residencia2'] = 'Casa/Governamental'
df.loc[ df['tipo_residencia'] == 'Com os pais', 'tipo_residencia2'] = 'Com os pais'
df.loc[ df['tipo_residencia'] == 'Aluguel', 'tipo_residencia2'] = 'Aluguel'
df.loc[ df['tipo_residencia'] == 'Estúdio', 'tipo_residencia2'] = 'Estúdio'

df[['tipo_residencia','tipo_residencia2']].head()

,tipo_residencia,tipo_residencia2
0,Casa,Casa/Governamental
1,Casa,Casa/Governamental
2,Casa,Casa/Governamental
3,Casa,Casa/Governamental
4,Governamental,Casa/Governamental


In [18]:
df['tipo_residencia2'].unique()

array(['Casa/Governamental', 'Com os pais', 'Aluguel', 'Estúdio', nan],
      dtype=object)

#### Vamos criar uma nova variável 'tipo_renda' e unificar as classificações Assalariado/Bolsista

In [19]:
df['tipo_renda'].unique()

array(['Empresário', 'Assalariado', 'Servidor público', 'Pensionista',
       'Bolsista'], dtype=object)

In [20]:
df.loc[ df['tipo_renda'] == 'Empresário', 'tipo_renda2'] ='Empresário'
df.loc[ df['tipo_renda'] == 'Assalariado', 'tipo_renda2'] = 'Assalariado/Bolsista'
df.loc[ df['tipo_renda'] == 'Servidor público', 'tipo_renda2'] = 'Servidor público'
df.loc[ df['tipo_renda'] == 'Pensionista', 'tipo_renda2'] = 'Pensionista'
df.loc[ df['tipo_renda'] == 'Bolsista', 'tipo_renda2'] = 'Assalariado/Bolsista'

df[['tipo_renda','tipo_renda2']].head()

,tipo_renda,tipo_renda2
0,Empresário,Empresário
1,Assalariado,Assalariado/Bolsista
2,Empresário,Empresário
3,Servidor público,Servidor público
4,Assalariado,Assalariado/Bolsista


In [21]:
df['tipo_renda2'].unique()

array(['Empresário', 'Assalariado/Bolsista', 'Servidor público',
       'Pensionista'], dtype=object)

### Criando a nova Design Matrix com o Patsy

In [22]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(0)) +  C(posse_de_imovel, Treatment(1)) + C(tipo_renda2, Treatment(0)) + C(educacao2, Treatment(2)) + C(tipo_residencia2, Treatment(1))', data = df)
x

DesignMatrix with shape (14936, 11)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(0))[T.M]',
     'C(posse_de_imovel, Treatment(1))[T.False]',
     'C(tipo_renda2, Treatment(0))[T.Empresário]',
     'C(tipo_renda2, Treatment(0))[T.Pensionista]',
     'C(tipo_renda2, Treatment(0))[T.Servidor público]',
     'C(educacao2, Treatment(2))[T.Primário/Secundário]',
     'C(educacao2, Treatment(2))[T.Superior completo/Pós graduação]',
     'C(tipo_residencia2, Treatment(1))[T.Aluguel]',
     'C(tipo_residencia2, Treatment(1))[T.Com os pais]',
     'C(tipo_residencia2, Treatment(1))[T.Estúdio]']
  Terms:
    'Intercept' (column 0)
    'C(sexo, Treatment(0))' (column 1)
    'C(posse_de_imovel, Treatment(1))' (column 2)
    'C(tipo_renda2, Treatment(0))' (columns 3:6)
    'C(educacao2, Treatment(2))' (columns 6:8)
    'C(tipo_residencia2, Treatment(1))' (columns 8:11)
  (to view full data, use np.asarray(this_obj))

#### Rodando o modelo e avaliando os parâmetros

In [23]:
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     284.9
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        12:00:02   Log-Likelihood:                -18036.
No. Observations:               14936   AIC:                         3.609e+04
Df Residuals:                   14925   BIC:                         3.618e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         7.8592      0.036    221.090      0.000       7.789       7.929
C(sexo, Treatment(0))[T.M]                                        0.6977      0.014     48.174      0.000       0.669       0.726
C(posse_de_imovel, Treatment(1))[T.False]                        -0.0894      0.014     -6.202      0.000      -0.118      -0.061
C(tipo_renda2, Treatment(0))[T.Empresário]                        0.0641      0.017      3.853      0.000       0.032       0.097
C(tipo_renda2, Treatment(0))[T.Pensionista]                      -0.1566      0.019     -8.236      0.000      -0.194      -0.119
C(tipo_renda2, Treatment(0))[T.Servidor público]                  0.2126      0.025      8.570      0.000       0.164       0.261
C(educacao2, Treatment(2))[T.Primário/Secundário]                 0.1240      0.035      3.533      0.000       0.055       0.193
C(educacao2, Treatment(2))[T.Superior completo/Pós graduação]     0.2178      0.036      6.096      0.000       0.148       0.288
C(tipo_residencia2, Treatment(1))[T.Aluguel]                     -0.1232      0.059     -2.098      0.036      -0.238      -0.008
C(tipo_residencia2, Treatment(1))[T.Com os pais]                 -0.1369      0.033     -4.213      0.000      -0.201      -0.073
C(tipo_residencia2, Treatment(1))[T.Estúdio]                      0.1907      0.089      2.134      0.033       0.015       0.366
==============================================================================
Omnibus:                      153.614   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              176.600
Skew:                           0.198   Prob(JB):                     4.49e-39
Kurtosis:                       3.357   Cond. No.                         18.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
R2_final = reg.rsquared

In [25]:
R2_ajust_final = reg.rsquared_adj

In [26]:
AIC_final = reg.aic
AIC_final

36094.6404507565

# Comparativo entre o modelo inicial (com todas as variáveis) e o final

In [27]:
R2_modelos = { 'Modelos': ['Modelo Inicial','Modelo Final'],
               'R2': [R2_inicial,R2_final],
              'R2_ajustado': [R2_ajust_inicial,R2_ajust_final],
              'AIC' : [AIC_inicial,AIC_final]
             }
df2= pd.DataFrame(R2_modelos)
df2

,Modelos,R2,R2_ajustado,AIC
0,Modelo Inicial,0.161934,0.160815,36241.337462
1,Modelo Final,0.160289,0.159726,36094.640451


Com a retirada das variáveis o modelo final teve uma pequena redução no R2, contudo, segundo o critério do AIC, houve uma pequena melhora no modelo devido a redução da complexidade.